In [17]:
from transformers import BertConfig, BertForTokenClassification, BertTokenizer
import torch
import time
import numpy as np

In [2]:
model_path = 'biobert_ner/pytorch_dumps/drug/'

In [3]:
tokenizer = BertTokenizer.from_pretrained(model_path, do_lower_case=False)
model = BertForTokenClassification.from_pretrained(model_path)

In [10]:
text = "Remdesivir 200mg IV on day 1, followed by 100 mg IV daily infusion for 9 days plus optimized supportive care or until discharge from hospital, whichever occurs first"
text_2 = "pyrazine carboxamide derivative (6-fluoro-3-hydroxy-2-pyrazinecarboxamide), a new type of RNA-dependent RNA polymerase (RdRp) inhibitor"

In [12]:
# inputs = tokenizer.convert_tokens_to_ids(tokenizer.tokenize(text))
# input_ids = get_token_ids(text, tokenizer)
input_ids = tokenizer([text, text_2], return_tensors="pt", padding=True)
# print(input_ids, len(input_ids))

In [13]:
input_ids

{'input_ids': tensor([[  101, 11336,  1306,  4704, 11083,  3161,  2363,  1306,  1403,  4191,
          1113,  1285,   122,   117,  1723,  1118,  1620, 17713,  4191,  3828,
          1107, 17149,  1111,   130,  1552,  4882, 11769,  3121, 26740, 17847,
          1920,  1137,  1235, 12398,  1121,  2704,   117,  1134, 17791,  4365,
          1148,   102,     0,     0,     0,     0,     0,     0,     0],
        [  101,   185, 20153, 25593,  1610,  8757, 11787,  2007, 14478,   113,
           127,   118, 23896, 14824,   118,   124,   118,   177, 19694, 16844,
           118,   123,   118,   185, 20153, 25593,  8766,  8757, 11787,  2007,
           114,   117,   170,  1207,  2076,  1104, 13254,   118,  7449, 13254,
         21176,  6530,   113, 23604,  2069,  1643,   114, 27558,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0],
        [0, 0

In [14]:
start = time.time()
outputs = model(**input_ids)
end = time.time()
print(end-start)

0.14939308166503906


In [20]:
logits = outputs[0].detach().numpy()

In [21]:
preds = np.argmax(logits,-1)

In [22]:
preds.shape

(2, 49)

In [23]:
print(preds)

[[5 1 4 4 4 4 3 4 4 3 3 3 3 3 3 3 3 3 3 3 3 4 3 3 3 3 3 4 4 3 3 3 3 3 3 3
  3 3 3 3 3 6 0 0 0 0 0 0 0]
 [5 1 4 4 2 4 4 4 3 3 1 2 2 4 2 2 2 2 4 4 2 2 2 2 4 4 4 4 4 4 3 3 3 3 3 3
  3 3 3 3 3 4 3 3 4 4 3 3 6]]


In [24]:
[5 1 4 4 2 4 4 4 3 3 1 2 2 4 2 2 2 2 4 4 2 2 2 2 4 4 4 4 4 4 3 3 3 3 3 3 3
 3 3 3 3 4 3 3 4 4 3 3 6]

SyntaxError: invalid syntax (<ipython-input-24-511de0f74a31>, line 1)

In [28]:
s_lens = []
for att_mask in input_ids['attention_mask']:
    index = 0
    found=False
    for index, val in enumerate(att_mask):
        if val==0:
            found=True
            break
    if not found:
        index+=1
    s_lens.append(index)

In [29]:
s_lens

[42, 49]

In [42]:
tot_tokens = []
for s_len, input_id in zip(s_lens, input_ids['input_ids']):
    tokens = tokenizer.convert_ids_to_tokens(input_id)
    tot_tokens += tokens[:s_len]
print(tot_tokens)

['[CLS]', 'Re', '##m', '##des', '##iv', '##ir', '200', '##m', '##g', 'IV', 'on', 'day', '1', ',', 'followed', 'by', '100', 'mg', 'IV', 'daily', 'in', '##fusion', 'for', '9', 'days', 'plus', 'op', '##ti', '##mized', 'supportive', 'care', 'or', 'until', 'discharge', 'from', 'hospital', ',', 'which', '##ever', 'occurs', 'first', '[SEP]', '[CLS]', 'p', '##yra', '##zine', 'car', '##box', '##ami', '##de', 'derivative', '(', '6', '-', 'flu', '##oro', '-', '3', '-', 'h', '##ydro', '##xy', '-', '2', '-', 'p', '##yra', '##zine', '##car', '##box', '##ami', '##de', ')', ',', 'a', 'new', 'type', 'of', 'RNA', '-', 'dependent', 'RNA', 'polymer', '##ase', '(', 'Rd', '##R', '##p', ')', 'inhibitor', '[SEP]']
